In [16]:
import pandas as pd
import argilla as rg

data_file = "data/data_etiquetada_attitude.pkl"

dataset = pd.read_pickle(data_file)



In [17]:
def to_conll_eval_format(data):
    output = []
    for d in data:
        sentence_tags = []
        for token in d['tokens']:
            sentence_tags.append(token['tag'])
        output.append(sentence_tags)
    return output

tokens = dataset["tokens"].tolist()
conll = to_conll_eval_format(dataset["metrics"].tolist())



In [18]:
# Iterate conll 

for text in conll:
    # Replace every "B-NG" in text (list) to 'O'
    for i, tag in enumerate(text):
        if tag == "B-NG":
            text[i] = "O"
        elif tag == "I-NG":
            text[i] = "O"


In [19]:
conll_total = [c for s in conll for c in s]
tags = list(set(conll_total))

# Remove "None" tag
tags.remove(None)
tags


['B-propriety (J3)',
 'I-Affect',
 'B-tenacity (J3)',
 'I-tenacity (J3)',
 'B-capacity (J3)',
 'I-Negative',
 'I-Social Sanction (J2)',
 'B-Social Sanction (J2)',
 'B-Social Esteem (J2)',
 'I-capacity (J3)',
 'I-normality (J3)',
 'B-normality (J3)',
 'B-Judgment (J1)',
 'B-Affect',
 'I-Judgment (J1)',
 'I-Appreciation',
 'B-Appreciation',
 'I-propriety (J3)',
 'I-veracity (J3)',
 'B-Negative',
 'I-Social Esteem (J2)',
 'B-veracity (J3)',
 'O']

In [20]:
import pandas as pd
data = pd.DataFrame({"tokens": tokens, "att_tags": conll})


In [21]:
# If any element in ng_tags is None, remove the whole row
data["has_none"] = data["att_tags"].apply(lambda x: None in x)

# Count has_none
print(data["has_none"].value_counts())

# Drop rows with has_none = True
data = data[data["has_none"] == False].reset_index(drop=True)

# Convert att_tags to string (all subelement in list)
data["att_tags"] = data["att_tags"].apply(lambda x: [str(elem) for elem in x])
# Convert tokens to strings
data["tokens"] = data["tokens"].apply(lambda x: [str(elem) for elem in x])

False    1354
True        4
Name: has_none, dtype: int64


In [22]:
data = data.reset_index()
# Rename index to id
data = data.rename(columns={"index": "id"})
#data = data.to_dict('records')


In [23]:
del data["has_none"]

In [24]:
# Len of dataset

# data["len_t"] = data["tokens"].apply(lambda x: len(x))
# data["len_ng"] = data["att_tags"].apply(lambda x: len(x))

# # Check if len_t == len_ng and count the number of sentences that are not equal
# data["len_equal"] = data["len_t"] == data["len_ng"]
# data["len_equal"].value_counts()

In [25]:
#features = Features({"att_tags": Sequence(ClassLabel(num_classes=3, names=['B-NG', 'I-NG', 'O']))})

#hf_dataset = Dataset.from_list(data, features=features)

In [26]:
from datasets import Dataset, Features, ClassLabel, Sequence, Value

hf_dataset = Dataset.from_pandas(data)

new_features = Features(
    {
        'id': Value(dtype='int64', id=None),
        'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
        'att_tags': Sequence(feature=ClassLabel(names=tags, id=None), length=-1, id=None),
    }
)

# Cast the dataset with the updated features.
hf_dataset = hf_dataset.cast(new_features)
hf_dataset

Casting the dataset:   0%|          | 0/2 [00:00<?, ?ba/s]

Dataset({
    features: ['id', 'tokens', 'att_tags'],
    num_rows: 1354
})

In [27]:
hf_dataset.features["att_tags"].feature.names


['B-propriety (J3)',
 'I-Affect',
 'B-tenacity (J3)',
 'I-tenacity (J3)',
 'B-capacity (J3)',
 'I-Negative',
 'I-Social Sanction (J2)',
 'B-Social Sanction (J2)',
 'B-Social Esteem (J2)',
 'I-capacity (J3)',
 'I-normality (J3)',
 'B-normality (J3)',
 'B-Judgment (J1)',
 'B-Affect',
 'I-Judgment (J1)',
 'I-Appreciation',
 'B-Appreciation',
 'I-propriety (J3)',
 'I-veracity (J3)',
 'B-Negative',
 'I-Social Esteem (J2)',
 'B-veracity (J3)',
 'O']

In [28]:
hf_dataset["att_tags"]

[[22,
  22,
  22,
  22,
  2,
  3,
  3,
  3,
  22,
  22,
  22,
  22,
  0,
  17,
  22,
  22,
  22,
  22,
  22,
  22,
  22,
  16,
  15,
  15,
  15,
  22],
 [4,
  9,
  9,
  9,
  9,
  9,
  9,
  22,
  22,
  22,
  22,
  22,
  22,
  16,
  15,
  15,
  15,
  15,
  15,
  22,
  22,
  22,
  22,
  22,
  22,
  22,
  22,
  22,
  22,
  22,
  22,
  22,
  22,
  22,
  22,
  22,
  22,
  22,
  22,
  2,
  3,
  22,
  22,
  22,
  22,
  22,
  22,
  22,
  22,
  22,
  22],
 [22,
  22,
  22,
  22,
  22,
  22,
  22,
  22,
  22,
  22,
  22,
  22,
  22,
  22,
  22,
  22,
  22,
  22,
  22,
  22,
  22,
  22,
  22,
  22,
  22,
  22,
  22,
  22,
  22,
  22,
  22,
  22,
  22,
  22,
  22,
  22,
  22,
  22,
  22,
  22,
  22,
  22,
  22,
  22,
  22,
  22,
  22,
  22,
  22,
  22,
  22,
  22,
  22,
  22,
  22,
  22,
  22,
  22,
  22,
  22,
  22,
  22,
  22,
  22,
  22,
  22,
  22,
  22,
  22,
  22,
  22,
  22,
  22,
  22,
  22,
  22,
  22,
  22,
  22,
  22,
  22,
  22,
  22,
  22,
  22,
  22,
  22,
  11,
  10,
  22,
  22,
  22

In [29]:
hf_dataset = hf_dataset.train_test_split(test_size=0.2, shuffle=True, seed=13)
# Rename datasets["test"] to datasets["validation"]
hf_dataset["validation"] = hf_dataset["test"]
del hf_dataset["test"]
hf_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'att_tags'],
        num_rows: 1083
    })
    validation: Dataset({
        features: ['id', 'tokens', 'att_tags'],
        num_rows: 271
    })
})

In [30]:
hf_dataset.push_to_hub("spanish_attitude_conll2003")

[06/18/23 14:43:48] WARNING  WARNING:datasets.dataset_dict:Pushing split train to the Hub.     ]8;id=455110;file:///home/jlortiz/data/anaconda3/envs/py310/lib/python3.10/site-packages/datasets/dataset_dict.py\dataset_dict.py]8;;\:]8;id=441576;file:///home/jlortiz/data/anaconda3/envs/py310/lib/python3.10/site-packages/datasets/dataset_dict.py#1532\1532]8;;\

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

Deleting unused files from dataset repository:   0%|          | 0/1 [00:00<?, ?it/s]

[06/18/23 14:44:08] WARNING  WARNING:datasets.dataset_dict:Pushing split validation to the     ]8;id=251525;file:///home/jlortiz/data/anaconda3/envs/py310/lib/python3.10/site-packages/datasets/dataset_dict.py\dataset_dict.py]8;;\:]8;id=401763;file:///home/jlortiz/data/anaconda3/envs/py310/lib/python3.10/site-packages/datasets/dataset_dict.py#1532\1532]8;;\
                             Hub.                                                                                  

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

Deleting unused files from dataset repository:   0%|          | 0/1 [00:00<?, ?it/s]